# Migración de Tests de Lectura
## Del formato anterior al nuevo formato simplificado

Este notebook convierte automáticamente los archivos JSON de tests del formato anterior al nuevo formato requerido por la aplicación Campayo simplificada.

In [1]:
import json
import os
from pathlib import Path
import re

# Configuración de carpetas
CARPETA_ORIGEN = "tests_antiguos"  # Carpeta con los JSONs del formato anterior
CARPETA_DESTINO = "tests_nuevos"   # Carpeta donde se guardarán los JSONs convertidos

# Crear carpetas si no existen
Path(CARPETA_ORIGEN).mkdir(exist_ok=True)
Path(CARPETA_DESTINO).mkdir(exist_ok=True)

print(f"Carpeta origen: {CARPETA_ORIGEN}")
print(f"Carpeta destino: {CARPETA_DESTINO}")

Carpeta origen: tests_antiguos
Carpeta destino: tests_nuevos


In [2]:
def limpiar_nombre_archivo(nombre):
    """
    Convierte nombres de archivos a formato válido para nombres de test.
    """
    # Remover extensión .json si existe
    nombre = re.sub(r'\.json$', '', nombre, flags=re.IGNORECASE)
    
    # Convertir a minúsculas y reemplazar espacios/caracteres especiales
    nombre = re.sub(r'[^a-zA-Z0-9_]', '_', nombre.lower())
    
    # Remover guiones bajos múltiples
    nombre = re.sub(r'_+', '_', nombre)
    
    # Remover guiones bajos al inicio y final
    nombre = nombre.strip('_')
    
    return nombre

def determinar_dificultad(test_data):
    """
    Determina la dificultad basándose en los datos del test anterior.
    """
    # Si tiene is_initial=True, es inicial
    if test_data.get('is_initial', False):
        return 'inicial'
    
    # Basarse en difficulty_level si existe
    difficulty_level = test_data.get('difficulty_level', '').lower()
    if difficulty_level in ['beginner', 'principiante']:
        return 'principiante'
    elif difficulty_level in ['intermediate', 'intermedio']:
        return 'intermedio'
    elif difficulty_level in ['advanced', 'avanzado']:
        return 'avanzado'
    elif difficulty_level == 'inicial':
        return 'inicial'
    
    # Basarse en difficulty_number si existe
    difficulty_number = test_data.get('difficulty_number', 1)
    if difficulty_number == 1:
        return 'inicial'
    elif difficulty_number <= 3:
        return 'principiante'
    elif difficulty_number <= 6:
        return 'intermedio'
    else:
        return 'avanzado'

def determinar_requisitos(test_data, nombre_test):
    """
    Determina los requisitos del test basándose en sus características.
    """
    test_previo = ""
    bloque_requerido = None
    
    # Si no es inicial, probablemente requiere el test inicial
    dificultad = determinar_dificultad(test_data)
    if dificultad != 'inicial' and nombre_test != 'inicial':
        test_previo = "inicial"
    
    # Determinar bloque requerido basándose en unlock_level
    unlock_level = test_data.get('unlock_level')
    if unlock_level is not None:
        if unlock_level <= 3:
            bloque_requerido = 1
        elif unlock_level <= 6:
            bloque_requerido = 2
        else:
            bloque_requerido = 3
    elif dificultad == 'intermedio':
        bloque_requerido = 1
    elif dificultad == 'avanzado':
        bloque_requerido = 2
    
    return test_previo, bloque_requerido

# Ejemplos de uso
print("Ejemplos de conversión de nombres:")
print(f"'Test Inicial.json' -> '{limpiar_nombre_archivo('Test Inicial.json')}'")
print(f"'20mil leguas.json' -> '{limpiar_nombre_archivo('20mil leguas.json')}'")
print(f"'test-avanzado-01.json' -> '{limpiar_nombre_archivo('test-avanzado-01.json')}'")

Ejemplos de conversión de nombres:
'Test Inicial.json' -> 'test_inicial'
'20mil leguas.json' -> '20mil_leguas'
'test-avanzado-01.json' -> 'test_avanzado_01'


In [3]:
def convertir_test(test_data_anterior, nombre_archivo):
    """
    Convierte un test del formato anterior al nuevo formato.
    """
    # Generar nombre limpio para el test
    nombre_test = limpiar_nombre_archivo(nombre_archivo)
    
    # Determinar requisitos
    test_previo, bloque_requerido = determinar_requisitos(test_data_anterior, nombre_test)
    
    # Estructura base del nuevo formato
    test_nuevo = {
        "nombre": nombre_test,
        "titulo": test_data_anterior.get('name', nombre_archivo),
        "descripcion": test_data_anterior.get('description', ''),
        "instrucciones": test_data_anterior.get('instructions', ''),
        "texto_titulo": test_data_anterior.get('text_title', ''),
        "texto_contenido": test_data_anterior.get('text_content', ''),
        "texto_autor": test_data_anterior.get('text_author', ''),
        "numero_palabras": 0,  # Se calculará automáticamente
        "dificultad": determinar_dificultad(test_data_anterior),
        "numero_test": test_data_anterior.get('difficulty_number', 1),
        "requiere_password": bool(test_data_anterior.get('unlock_code')),
        "password_acceso": test_data_anterior.get('unlock_code', ''),
        "requiere_pro": False,  # Por defecto False, ajustar manualmente si es necesario
        "test_previo_requerido": test_previo,
        "bloque_requerido": bloque_requerido,
        "activo": True,
        "preguntas": []
    }
    
    # Convertir preguntas
    preguntas_anteriores = test_data_anterior.get('questions', [])
    
    for pregunta_anterior in preguntas_anteriores:
        pregunta_nueva = {
            "pregunta": pregunta_anterior.get('question', ''),
            "opciones": []
        }
        
        # Convertir opciones
        opciones_anteriores = pregunta_anterior.get('options', [])
        
        for opcion_anterior in opciones_anteriores:
            opcion_nueva = {
                "texto": opcion_anterior.get('text', ''),
                "es_correcta": opcion_anterior.get('is_correct', False)
            }
            pregunta_nueva["opciones"].append(opcion_nueva)
        
        test_nuevo["preguntas"].append(pregunta_nueva)
    
    return test_nuevo

# Función de validación
def validar_test_convertido(test_data):
    """
    Valida que el test convertido tenga todos los campos requeridos.
    """
    errores = []
    
    # Campos requeridos
    campos_requeridos = ['nombre', 'titulo', 'descripcion', 'instrucciones', 
                        'texto_titulo', 'texto_contenido', 'dificultad', 'numero_test']
    
    for campo in campos_requeridos:
        if not test_data.get(campo):
            errores.append(f"Campo requerido vacío: {campo}")
    
    # Validar preguntas
    preguntas = test_data.get('preguntas', [])
    if not preguntas:
        errores.append("No hay preguntas en el test")
    
    for i, pregunta in enumerate(preguntas):
        if not pregunta.get('pregunta'):
            errores.append(f"Pregunta {i+1}: texto de pregunta vacío")
        
        opciones = pregunta.get('opciones', [])
        if len(opciones) < 2:
            errores.append(f"Pregunta {i+1}: menos de 2 opciones")
        
        opciones_correctas = sum(1 for opt in opciones if opt.get('es_correcta'))
        if opciones_correctas != 1:
            errores.append(f"Pregunta {i+1}: debe tener exactamente 1 opción correcta (tiene {opciones_correctas})")
    
    return errores

print("Funciones de conversión y validación definidas.")

Funciones de conversión y validación definidas.


In [4]:
# Buscar todos los archivos JSON en la carpeta origen
archivos_json = list(Path(CARPETA_ORIGEN).glob("*.json"))

print(f"Archivos JSON encontrados en {CARPETA_ORIGEN}:")
for archivo in archivos_json:
    print(f"  - {archivo.name}")

if not archivos_json:
    print("\n⚠️  No se encontraron archivos JSON en la carpeta origen.")
    print(f"   Asegúrate de colocar los archivos JSON en la carpeta: {CARPETA_ORIGEN}")
else:
    print(f"\n✅ {len(archivos_json)} archivo(s) encontrado(s) para procesar.")

Archivos JSON encontrados en tests_antiguos:
  - 20mil.json
  - 3mosq.json
  - caballerooxidada.json
  - corsario.json
  - crusoe.json
  - gama.json
  - islatesoro.json
  - moby.json
  - principito.json
  - sawyer.json
  - test_inicial.json
  - tortuga.json
  - yacare.json

✅ 13 archivo(s) encontrado(s) para procesar.


In [6]:
# Procesar cada archivo JSON
resultados = {
    'exitosos': [],
    'errores': [],
    'advertencias': []
}

for archivo_origen in archivos_json:
    print(f"\n📄 Procesando: {archivo_origen.name}")
    
    try:
        # Leer archivo original
        with open(archivo_origen, 'r', encoding='utf-8') as f:
            test_anterior = json.load(f)
        
        print(f"   ✓ Archivo leído correctamente")
        
        # Convertir al nuevo formato
        test_nuevo = convertir_test(test_anterior, archivo_origen.stem)
        
        print(f"   ✓ Convertido al nuevo formato")
        print(f"     - Nombre del test: {test_nuevo['nombre']}")
        print(f"     - Dificultad: {test_nuevo['dificultad']}")
        print(f"     - Número de preguntas: {len(test_nuevo['preguntas'])}")
        
        # Validar test convertido
        errores_validacion = validar_test_convertido(test_nuevo)
        
        if errores_validacion:
            print(f"   ⚠️  Advertencias de validación:")
            for error in errores_validacion:
                print(f"      - {error}")
            resultados['advertencias'].append({
                'archivo': archivo_origen.name,
                'test_nombre': test_nuevo['nombre'],
                'errores': errores_validacion
            })
        
        # Generar nombre de archivo destino
        nombre_destino = f"{test_nuevo['nombre']}.json"
        archivo_destino = Path(CARPETA_DESTINO) / nombre_destino
        
        # Guardar archivo convertido
        with open(archivo_destino, 'w', encoding='utf-8') as f:
            json.dump(test_nuevo, f, indent=2, ensure_ascii=False)
        
        print(f"   ✅ Guardado como: {nombre_destino}")
        
        resultados['exitosos'].append({
            'archivo_origen': archivo_origen.name,
            'archivo_destino': nombre_destino,
            'test_nombre': test_nuevo['nombre'],
            'num_preguntas': len(test_nuevo['preguntas'])
        })
        
    except json.JSONDecodeError as e:
        error_msg = f"Error al leer JSON: {str(e)}"
        print(f"   ❌ {error_msg}")
        resultados['errores'].append({
            'archivo': archivo_origen.name,
            'error': error_msg
        })
        
    except Exception as e:
        error_msg = f"Error inesperado: {str(e)}"
        print(f"   ❌ {error_msg}")
        resultados['errores'].append({
            'archivo': archivo_origen.name,
            'error': error_msg
        })


📄 Procesando: 20mil.json
   ✓ Archivo leído correctamente
   ✓ Convertido al nuevo formato
     - Nombre del test: 20mil
     - Dificultad: intermedio
     - Número de preguntas: 20
   ✅ Guardado como: 20mil.json

📄 Procesando: 3mosq.json
   ✓ Archivo leído correctamente
   ✓ Convertido al nuevo formato
     - Nombre del test: 3mosq
     - Dificultad: avanzado
     - Número de preguntas: 20
   ✅ Guardado como: 3mosq.json

📄 Procesando: caballerooxidada.json
   ✓ Archivo leído correctamente
   ✓ Convertido al nuevo formato
     - Nombre del test: caballerooxidada
     - Dificultad: intermedio
     - Número de preguntas: 20
   ✅ Guardado como: caballerooxidada.json

📄 Procesando: corsario.json
   ✓ Archivo leído correctamente
   ✓ Convertido al nuevo formato
     - Nombre del test: corsario
     - Dificultad: intermedio
     - Número de preguntas: 20
   ✅ Guardado como: corsario.json

📄 Procesando: crusoe.json
   ✓ Archivo leído correctamente
   ✓ Convertido al nuevo formato
     - Nomb

In [7]:
# Mostrar resumen de resultados
print("\n" + "="*60)
print("📊 RESUMEN DE CONVERSIÓN")
print("="*60)

print(f"\n✅ ARCHIVOS CONVERTIDOS EXITOSAMENTE: {len(resultados['exitosos'])}")
for resultado in resultados['exitosos']:
    print(f"   📄 {resultado['archivo_origen']} → {resultado['archivo_destino']}")
    print(f"      Test: {resultado['test_nombre']} ({resultado['num_preguntas']} preguntas)")

if resultados['advertencias']:
    print(f"\n⚠️  ARCHIVOS CON ADVERTENCIAS: {len(resultados['advertencias'])}")
    for advertencia in resultados['advertencias']:
        print(f"   📄 {advertencia['archivo']} (test: {advertencia['test_nombre']})")
        for error in advertencia['errores']:
            print(f"      - {error}")

if resultados['errores']:
    print(f"\n❌ ARCHIVOS CON ERRORES: {len(resultados['errores'])}")
    for error in resultados['errores']:
        print(f"   📄 {error['archivo']}: {error['error']}")

print(f"\n📁 Los archivos convertidos están en la carpeta: {CARPETA_DESTINO}")
print("\n📋 SIGUIENTE PASO:")
print("   Copia los archivos JSON convertidos a la carpeta:")
print("   management/commands/tests_lectura/")
print("   y ejecuta: python manage.py preparar_datos")


📊 RESUMEN DE CONVERSIÓN

✅ ARCHIVOS CONVERTIDOS EXITOSAMENTE: 13
   📄 20mil.json → 20mil.json
      Test: 20mil (20 preguntas)
   📄 3mosq.json → 3mosq.json
      Test: 3mosq (20 preguntas)
   📄 caballerooxidada.json → caballerooxidada.json
      Test: caballerooxidada (20 preguntas)
   📄 corsario.json → corsario.json
      Test: corsario (20 preguntas)
   📄 crusoe.json → crusoe.json
      Test: crusoe (20 preguntas)
   📄 gama.json → gama.json
      Test: gama (20 preguntas)
   📄 islatesoro.json → islatesoro.json
      Test: islatesoro (20 preguntas)
   📄 moby.json → moby.json
      Test: moby (20 preguntas)
   📄 principito.json → principito.json
      Test: principito (20 preguntas)
   📄 sawyer.json → sawyer.json
      Test: sawyer (20 preguntas)
   📄 test_inicial.json → test_inicial.json
      Test: test_inicial (20 preguntas)
   📄 tortuga.json → tortuga.json
      Test: tortuga (20 preguntas)
   📄 yacare.json → yacare.json
      Test: yacare (20 preguntas)

📁 Los archivos convertido

## Verificación Manual

Puedes verificar manualmente los archivos convertidos:

In [8]:
# Mostrar ejemplo de conversión (primer archivo exitoso)
if resultados['exitosos']:
    primer_exitoso = resultados['exitosos'][0]
    archivo_ejemplo = Path(CARPETA_DESTINO) / primer_exitoso['archivo_destino']
    
    print(f"📄 Ejemplo de archivo convertido: {archivo_ejemplo.name}")
    print("\n" + "-"*50)
    
    with open(archivo_ejemplo, 'r', encoding='utf-8') as f:
        test_ejemplo = json.load(f)
    
    # Mostrar estructura principal
    campos_principales = ['nombre', 'titulo', 'dificultad', 'numero_test', 
                         'test_previo_requerido', 'bloque_requerido']
    
    for campo in campos_principales:
        valor = test_ejemplo.get(campo)
        print(f"{campo}: {valor}")
    
    print(f"\nNúmero de preguntas: {len(test_ejemplo.get('preguntas', []))}")
    
    # Mostrar primera pregunta como ejemplo
    if test_ejemplo.get('preguntas'):
        primera_pregunta = test_ejemplo['preguntas'][0]
        print(f"\nEjemplo de pregunta:")
        print(f"Pregunta: {primera_pregunta['pregunta'][:100]}...")
        print(f"Opciones: {len(primera_pregunta['opciones'])}")
        
        opciones_correctas = sum(1 for opt in primera_pregunta['opciones'] if opt['es_correcta'])
        print(f"Opciones correctas: {opciones_correctas}")
else:
    print("No hay archivos convertidos exitosamente para mostrar como ejemplo.")

📄 Ejemplo de archivo convertido: 20mil.json

--------------------------------------------------
nombre: 20mil
titulo: Veinte mil leguas de viaje submarino
dificultad: intermedio
numero_test: 5
test_previo_requerido: inicial
bloque_requerido: 1

Número de preguntas: 20

Ejemplo de pregunta:
Pregunta: ¿Cuál es la ubicación geográfica de la isla Crespo según el texto?...
Opciones: 4
Opciones correctas: 1


## Notas Importantes

### Campos que podrían necesitar ajuste manual:

1. **`requiere_pro`**: Por defecto se establece en `false`. Ajustar manualmente para tests que requieran plan Pro.

2. **`test_previo_requerido`**: Se asigna automáticamente "inicial" para tests no iniciales. Verificar si es correcto.

3. **`bloque_requerido`**: Se asigna basándose en la dificultad. Verificar si es correcto para cada test.

4. **`numero_test`**: Se toma del campo `difficulty_number` anterior. Verificar la secuencia.

### Validaciones realizadas:

- ✅ Todos los campos requeridos están presentes
- ✅ Cada pregunta tiene texto
- ✅ Cada pregunta tiene al menos 2 opciones
- ✅ Cada pregunta tiene exactamente 1 opción correcta
- ✅ Estructura JSON válida